In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
# 全量数据

train_data = pd.read_csv('./data_format1/train_format1.csv')
test_data = pd.read_csv('./data_format1/test_format1.csv')
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
user_log = pd.read_csv('./data_format1/user_log_format1.csv')


In [5]:
# 加载样本
'''
train_data = pd.read_csv('./sample/train_data_sample.csv')
test_data = pd.read_csv('./sample/test_data_sample.csv')
user_info = pd.read_csv('./sample/user_info_sample.csv')
user_log = pd.read_csv('./sample/user_log_sample.csv')
'''

"\ntrain_data = pd.read_csv('./sample/train_data_sample.csv')\ntest_data = pd.read_csv('./sample/test_data_sample.csv')\nuser_info = pd.read_csv('./sample/user_info_sample.csv')\nuser_log = pd.read_csv('./sample/user_log_sample.csv')\n"

In [3]:
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

In [4]:
# 数据清洗
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int64')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
user_info['age_range'].fillna(0, inplace=True)
user_info = user_info.replace([8], [7])
user_info['age_range'] = user_info['age_range'].astype('int64')
user_info['gender'].fillna(2, inplace=True)
user_info['gender'] = user_info['gender'].astype('int64')

In [5]:
print(train_data.info())
print(user_info.info())
print(user_log.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    424170 non-null  int64
 1   age_range  424170 non-null  int64
 2   gender     424170 non-null  int64
dtypes: int64(3)
memory usage: 9.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Non-Null Count     Dtype         
---  ------       --------------     -----         
 0   user_id      54925330 non-null  int64         
 1   item_id      549253

In [6]:
train_data['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data, test_data], ignore_index=True, sort=False)
matrix.drop('prob', axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

In [7]:
# 用户特征处理
groups = user_log.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0: 'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'u7', 1: 'u8', 2: 'u9', 3: 'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 商户特征处理
groups = user_log.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'm1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id': 'm2', 'item_id': 'm3', 'cat_id': 'm4', 'brand_id': 'm5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'm6', 1: 'm7', 2: 'm8', 3: 'm9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = train_data[train_data['label'] == -1].groupby('merchant_id').size().reset_index().rename(columns={0: 'm10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [8]:
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id': 'um2', 'cat_id': 'um3', 'brand_id': 'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'um5', 1: 'um6', 2: 'um7', 3: 'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['last', 'first'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

/Users/paulhwang/anaconda3/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [9]:
matrix['r1'] = matrix['u9'] / matrix['u7']
matrix['r2'] = matrix['m8'] / matrix['m6']
matrix['r3'] = matrix['um7'] / matrix['um5']
matrix.fillna(0, inplace=True)
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='gen')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [15]:
categorical_features_indices = ['u7', 'u8', 'u9', 'u10', 'm6', 'm7', 'm8', 'm9', 'um5', 'um6', 'um7', 'um8', 'age_0.0', 'age_1.0', 'age_2.0', 'age_3.0', 'age_4.0',
                               'age_5.0', 'age_6.0', 'age_7.0', 'gen_0.0', 'gen_1.0', 'gen_2.0']
for i in categorical_features_indices:
    matrix[i] = matrix[i].astype('int64')

train_data = matrix[matrix['origin'] == 'train'].drop('origin', axis=1)
test = matrix[matrix['origin'] == 'test'].drop(['origin', 'label'], axis=1)
x_train, y_train = train_data.drop('label', axis=1), train_data['label']

In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260865 entries, 0 to 260864
Data columns (total 45 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      260865 non-null  int64  
 1   merchant_id  260865 non-null  int64  
 2   u1           260865 non-null  int64  
 3   u2           260865 non-null  int64  
 4   u3           260865 non-null  int64  
 5   u4           260865 non-null  int64  
 6   u5           260865 non-null  int64  
 7   u6           260865 non-null  float64
 8   u7           260865 non-null  float64
 9   u8           260865 non-null  float64
 10  u9           260865 non-null  float64
 11  u10          260865 non-null  float64
 12  m1           260865 non-null  int64  
 13  m2           260865 non-null  int64  
 14  m3           260865 non-null  int64  
 15  m4           260865 non-null  int64  
 16  m5           260865 non-null  int64  
 17  m6           260865 non-null  float64
 18  m7           260865 non-

In [27]:
train_x, vali_x, train_y, vali_y = train_test_split(x_train, y_train, test_size=0.2)
model = CatBoostClassifier(iterations=1500, depth=6, learning_rate=0.015, loss_function='Logloss', eval_metric='AUC', bagging_temperature=0.83, rsm=0.78, od_type='Iter',
                          od_wait=150, metric_period=50, l2_leaf_reg=5, thread_count=20, random_seed=967, logging_level='Verbose')

model.fit(train_x, train_y, eval_set=(vali_x, vali_y), cat_features=categorical_features_indices)

0:	test: 0.5742304	best: 0.5742304 (0)	total: 140ms	remaining: 3m 30s
50:	test: 0.6259265	best: 0.6265540 (41)	total: 6.58s	remaining: 3m 7s
100:	test: 0.6660367	best: 0.6660367 (100)	total: 14.7s	remaining: 3m 23s
150:	test: 0.6702016	best: 0.6702016 (150)	total: 25.7s	remaining: 3m 49s
200:	test: 0.6724564	best: 0.6724661 (198)	total: 36.9s	remaining: 3m 58s
250:	test: 0.6747086	best: 0.6747086 (250)	total: 47.7s	remaining: 3m 57s
300:	test: 0.6763149	best: 0.6763170 (298)	total: 58.5s	remaining: 3m 53s
350:	test: 0.6774276	best: 0.6774276 (350)	total: 1m 7s	remaining: 3m 41s
400:	test: 0.6780672	best: 0.6780672 (400)	total: 1m 18s	remaining: 3m 35s
450:	test: 0.6787507	best: 0.6787507 (450)	total: 1m 29s	remaining: 3m 29s
500:	test: 0.6793446	best: 0.6793446 (500)	total: 1m 40s	remaining: 3m 20s
550:	test: 0.6798645	best: 0.6798645 (550)	total: 1m 51s	remaining: 3m 11s
600:	test: 0.6802855	best: 0.6802875 (598)	total: 2m 1s	remaining: 3m 2s
650:	test: 0.6806828	best: 0.6806828 (650)

In [28]:
prob = model.predict_proba(test)
test_data['prob'] = pd.Series(prob[:, 1])
#test_data.drop(['origin'], axis=1, inplace=True)

In [29]:
test_data.to_csv('prediction_catboost.csv', index=False)

In [19]:
test_data

,user_id,merchant_id,prob
0,163968,4605,0.055029
1,360576,1581,0.097740
2,98688,1964,0.055483
3,98688,3645,0.050062
4,295296,3361,0.063816
...,...,...,...
261472,228479,3111,0.079313
261473,97919,2341,0.029915
261474,97919,3971,0.073271
261475,32639,3536,0.047449
